In [1]:
import pandas as pd
import datetime
import networkx as nx
import sys
import time as tm
import numpy as np

In [2]:
DAY_SECONDS = 86400
HOUR_SECONDS = 3600 
MINUTE_SECONDS = 60



# Time format conversion functions in the format HH:MM:SS
def fix_time(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    hours = hours % 24
    return f'{hours:02d}:{minutes:02d}:{seconds:02d}'

def time_to_seconds(time):
    return time.second + time.minute*60 + time.hour*3600

def seconds_to_hour_and_minute(seconds):
	return "hours: ", seconds // HOUR_SECONDS,"minutes: ", (seconds % HOUR_SECONDS) // MINUTE_SECONDS

In [3]:
# Import data
df = pd.read_csv("connection_graph.csv", low_memory=False)
df.head()

,Unnamed: 0,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383


In [4]:
# Clean data
df.rename(columns={df.columns[0]: 'del'}, inplace=True)

df.drop(columns=['del','company'], inplace=True)
df.drop_duplicates(inplace=True)
df = df[df['start_stop'] != df['end_stop']]

# Fix time format
df['departure_time'] = df['departure_time'].apply(fix_time)
df['arrival_time'] = df['arrival_time'].apply(fix_time)
df['departure_time'] = pd.to_datetime(df['departure_time'], format='%H:%M:%S').dt.time
df['arrival_time'] = pd.to_datetime(df['arrival_time'], format='%H:%M:%S').dt.time

# Add time interpretation in seconds
df['departure_time_seconds'] = df['departure_time'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)
df['arrival_time_seconds'] = df['arrival_time'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)

df['id'] = range(0, len(df))
df.set_index('id', inplace=True)

df.head()

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_time_seconds,arrival_time_seconds
id,,,,,,,,,,,
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,75120,75180
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,75180,75240
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,75240,75300
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,75300,75420
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,75420,75540


In [5]:
# Now we need to prepare the list of stops we are working with, dropping the duplicates to provide data integrity
stops = df[['start_stop', 'start_stop_lat', 'start_stop_lon']].drop_duplicates(subset=['start_stop'], keep='first').sort_values(by='start_stop')

# No longer need for lat and lon info after getting stops so we need to get rid of them in main data
df.drop(columns=['start_stop_lat','start_stop_lon', 'end_stop_lat','end_stop_lon'], inplace=True)
print(len(stops))
stops.head()
df.head()

938


,line,departure_time,arrival_time,start_stop,end_stop,departure_time_seconds,arrival_time_seconds
id,,,,,,,
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,75120,75180
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),75180,75240
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,75240,75300
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,75300,75420
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,75420,75540


In [6]:
# Graph creation
stop_graph = nx.DiGraph()

def create_stop_nodes(conn):
    stop_graph.add_node(conn['start_stop'], lat=conn['start_stop_lat'], lon=conn['start_stop_lon'])

stops.apply(create_stop_nodes, axis=1)

print(len(stop_graph.nodes))
print(list(stop_graph.nodes.data())[None:10:None])

938
[('8 Maja', {'lat': 51.11366919, 'lon': 17.09111971}), ('AUCHAN', {'lat': 51.0520648, 'lon': 16.97405354}), ('Adamczewskich', {'lat': 51.12084396, 'lon': 16.86953486}), ('Adamieckiego', {'lat': 51.07425917, 'lon': 16.96299503}), ('Aleja Architektów', {'lat': 51.13881871, 'lon': 16.92905866}), ('Aleja Pracy', {'lat': 51.0918696, 'lon': 16.98537334}), ('Aleja Wędrowców', {'lat': 51.0879861, 'lon': 16.99188243}), ('Arabska', {'lat': 51.05115765, 'lon': 17.0923256}), ('Arachidowa', {'lat': 51.15417321, 'lon': 16.89901728}), ('Arena', {'lat': 51.094134, 'lon': 17.026404})]


In [7]:
# Adding edges
def create_edges(conn):
    if not stop_graph.has_edge(conn['start_stop'], conn['end_stop']):
        stop_graph.add_edge(conn['start_stop'], conn['end_stop'], schedule=[])

    # Append the new connection data to the schedule list
    stop_graph[conn['start_stop']][conn['end_stop']]['schedule'].append(conn.to_dict())
    
df.apply(create_edges, axis=1)
print(stop_graph['Ostrowskiego']['FAT']['schedule'][0])

{'line': '107', 'departure_time': datetime.time(7, 11), 'arrival_time': datetime.time(7, 12), 'start_stop': 'Ostrowskiego', 'end_stop': 'FAT', 'departure_time_seconds': 25860, 'arrival_time_seconds': 25920}


In [8]:
# Function to retrieve path
def return_path_to_destination(start, end, line, best_routes_df):
    path = []
    (line_info, when, stop) = (f"leave line: { line[end]['line']}", line[end]['arrival_time'], f"at: {end}")
    path.append((line_info, when, stop))
    end = best_routes_df.loc['previous_stop', end]
    current_line = line[end]['line']
    while end != start:
        previous_stop_info = line[best_routes_df.loc['previous_stop', end]]

        if previous_stop_info is not None and previous_stop_info['line'] != current_line:
            (line_info, when, stop) = (f"enter line: { line[end]['line']}", line[end]['departure_time'], f"at: {line[end]['start_stop']}")
            path.append((line_info, when, stop))

            tmp_stop = best_routes_df.loc['previous_stop', end]
            (line_info, when, stop) = (f"leave line: { line[tmp_stop]['line']}", line[tmp_stop]['arrival_time'], f"at: {tmp_stop}")
            path.append((line_info, when, stop))

        future_end = best_routes_df.loc['previous_stop', end]

        if future_end != start:
            end = future_end
            current_line = line[end]['line']
        else:
            (line_info, when, stop) = (f"enter line: { line[end]['line']}", line[end]['departure_time'], f"at: {line[end]['start_stop']}")
            path.append((line_info, when, stop))
            end = future_end
    return path

def print_path(path):
    for i in range(len(path) - 1, -1, -1):
        print(path[i])

# Cost of travel's time
def count_total_cost(arrival_time_seconds, start_time_seconds):
    return (arrival_time_seconds - start_time_seconds + DAY_SECONDS) % DAY_SECONDS

def min_cost(unchecked_stops, best_routes_df):
    min_cost = best_routes_df.loc['min_cost', unchecked_stops[0]]
    min_stop = unchecked_stops[0]

    for stop in unchecked_stops:
        if best_routes_df.loc['min_cost', stop] < min_cost:
            min_cost = best_routes_df.loc['min_cost', stop]
            min_stop = stop
    return min_stop

def best_connection(schedule, current_time_seconds, current_cost):
    best_con = None
    best_cost = float('inf')

    for connection in schedule:
        waiting_time = (connection['departure_time_seconds'] - current_time_seconds + DAY_SECONDS) % DAY_SECONDS
        travel_time = ((connection['arrival_time_seconds'] - connection['departure_time_seconds'] + DAY_SECONDS) % DAY_SECONDS)
        total_cost = current_cost + waiting_time + travel_time

        if best_cost > total_cost:
            best_cost = total_cost
            best_con = connection

    return (best_con, best_cost)


In [9]:
from math import sqrt

average_speed_mpk_km_s = 17 / 3600  # average velocity in Wrocław [km/s]
degrees_to_km = 111 

def best_connection_min_time(schedule, current_time_seconds, current_cost, previous_connection=None):
    best_connection = None
    best_cost = float('inf')

    for connection in schedule:
        waiting_time = (connection['departure_time_seconds'] - current_time_seconds + DAY_SECONDS) % DAY_SECONDS
        travel_time = (connection['arrival_time_seconds'] - connection['departure_time_seconds'] + DAY_SECONDS) % DAY_SECONDS
        total_cost = current_cost + waiting_time + travel_time

        if total_cost < best_cost:
            best_cost = total_cost
            best_connection = connection

    return (best_connection, best_cost)

def best_connection_min_transfers(schedule, current_time_seconds, current_cost, previous_connection):
    previous_line = ""
    if previous_connection:
        previous_line = previous_connection['line']
    best_connection = None
    best_cost = float('inf')

    for connection in schedule:
        waiting_time = (connection['departure_time_seconds'] - current_time_seconds + 86400) % 86400
        travel_time = ((connection['arrival_time_seconds'] - connection['departure_time_seconds'] + 86400) % 86400)
        total_cost = current_cost + waiting_time + travel_time

        if previous_line and connection['line'] != previous_line:
            total_cost += 1000  # Penalizing for a line change

        if best_cost > total_cost:
            best_cost = total_cost
            best_connection = connection

    return (best_connection, best_cost)

def h_manhattan(current_node, goal_node):
    # Calculating Manhattan distance based on latitude and longitude
    distance_km = degrees_to_km * sqrt((current_node['lat'] - goal_node['lat'])**2 + (current_node['lon'] - goal_node['lon'])**2)
    return distance_km / average_speed_mpk_km_s

def h_euclidean(current_node, goal_node):
    # Calculating Euclidean distance for an approximation
    distance_km = degrees_to_km * (abs(current_node['lat'] - goal_node['lat']) + abs(current_node['lon'] - goal_node['lon']))
    return distance_km / average_speed_mpk_km_s

def haversine(current_node, goal_node):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [current_node['lat'], current_node['lon'], goal_node['lat'], goal_node['lon']])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return distance / average_speed_mpk_km_s

def astar(graph, start, end, time, h=h_euclidean, best_connection_function=best_connection_min_transfers):
    algorithm_start_time = tm.time()

    # Initialize all nodes in the graph with default g, h, and f values
    for node in graph.nodes():
        graph.nodes[node]['g'] = 0  # Cost from start to node
        graph.nodes[node]['h'] = 0  # Heuristic estimate from node to end
        graph.nodes[node]['f'] = 0  # Total cost (g + h)

    # Creating DataFrame with unique stops as columns to store the outcomes
    best_routes_df = pd.DataFrame(columns=graph.nodes)
    best_routes_df.loc['min_cost'] = float('inf')  # Initialize min_cost for all nodes
    best_routes_df.loc['previous_stop'] = ""  # To track the best path
    best_routes_df.loc['arrival_time_seconds'] = None  # Store arrival times for best paths

    # Setting initial values
    current_time_seconds = time_to_seconds(time)
    best_routes_df.loc['min_cost', start] = 0  # Start node has a cost of 0 to reach
    best_routes_df.loc['best_arrival_time_seconds', start] = current_time_seconds

    # Lists to track which stops have been checked or need to be checked
    checked_stops = []  # Closed set
    unchecked_stops = list(graph.nodes)  # Open set, initially contains all nodes

    # Creating a dictionary to store the best connection for each stop
    line = {node: None for node in graph.nodes}

    while unchecked_stops:
        current_stop = None
        lowest_cost = float('inf')
        
        # Find the stop with the lowest f value (min_cost) among the unchecked stops
        current_stop = min_cost(unchecked_stops, best_routes_df)
        lowest_cost = best_routes_df.loc['min_cost', current_stop]
        current_time_seconds = best_routes_df.loc['best_arrival_time_seconds', current_stop]

        # Break if the end stop is reached
        if current_stop == end:
            break
        
        # Remove the current stop from unchecked_stops and add it to checked_stops
        unchecked_stops.remove(current_stop)
        checked_stops.append(current_stop)

        # Explore all neighbors of the current stop
        for neighbor in graph.neighbors(current_stop):
            connection, cost = best_connection_function(graph[current_stop][neighbor]['schedule'], current_time_seconds, lowest_cost, line[current_stop])

            # If a better path to the neighbor is found, update the best routes dataframe
            if best_routes_df.loc['min_cost', neighbor] > cost:
                best_routes_df.loc['min_cost', neighbor] = cost
                best_routes_df.loc['previous_stop', neighbor] = current_stop
                best_routes_df.loc['best_arrival_time_seconds', neighbor] = connection['arrival_time_seconds']
                line[neighbor] = connection

            # Update heuristic and total cost values for neighbors
            if neighbor not in unchecked_stops and neighbor not in checked_stops:
                unchecked_stops.append(neighbor)
                graph.nodes[neighbor]['h'] = h(graph.nodes[neighbor], graph.nodes[end])
                graph.nodes[neighbor]['g'] = graph.nodes[current_stop]['g'] + cost
                graph.nodes[neighbor]['f'] = graph.nodes[neighbor]['g'] + graph.nodes[neighbor]['h']
            elif graph.nodes[neighbor]['g'] > graph.nodes[current_stop]['g'] + cost:
                graph.nodes[neighbor]['g'] = graph.nodes[current_stop]['g'] + cost
                graph.nodes[neighbor]['f'] = graph.nodes[neighbor]['g'] + graph.nodes[neighbor]['h']
                if neighbor in checked_stops:
                    unchecked_stops.append(neighbor)
                    checked_stops.remove(neighbor)

    # Calculate the total duration of the algorithm execution
    algorithm_duration = tm.time() - algorithm_start_time
    return (line, best_routes_df, algorithm_duration)


In [10]:
def get_route_details(solution, initial_stop, departure_hour, transport_graph, connection_strategy):
	route_sequence = tuple(solution)
	connection_lines_details = []
	astar_routes_dataframes = []
	
	current_time = departure_hour
	current_stop = initial_stop
	
	route_with_start = solution.copy()
	route_with_start.append(initial_stop)

	total_travel_cost = 0

	for next_stop in route_with_start:
		(optimal_route, astar_route_df, route_calculation_time) = astar(transport_graph, current_stop, next_stop, current_time, h_euclidean, connection_strategy)
		leg_cost = count_total_cost(optimal_route[next_stop]['arrival_time_seconds'], time_to_seconds(current_time))
		total_travel_cost += leg_cost
		current_time = optimal_route[next_stop]['arrival_time']
		current_stop = next_stop
		connection_lines_details.append(optimal_route)
		astar_routes_dataframes.append(astar_route_df)

	return (total_travel_cost, connection_lines_details, astar_routes_dataframes)


In [11]:
import random

# Mapping of solutions to their total travel costs
solution_costs = {}

def evaluate_solution_cost(route, initial_stop, departure_time, transport_network, route_selector):
    route_key = tuple(route)

    # Return the cached cost if the solution has been evaluated before
    if route_key in solution_costs:
        return solution_costs[route_key]

    current_time = departure_time
    current_stop = initial_stop
    route_extended = route.copy()
    route_extended.append(initial_stop)

    total_cost = 0

    # Calculate the cost for the solution by iterating through stops
    for next_stop in route_extended:
        (optimal_route, _, _) = astar(transport_network, current_stop, next_stop, current_time, h_euclidean, route_selector)
        leg_cost = count_total_cost(optimal_route[next_stop]['arrival_time_seconds'], time_to_seconds(current_time))
        total_cost += leg_cost
        current_time = optimal_route[next_stop]['arrival_time']
        current_stop = next_stop

    solution_costs[route_key] = total_cost
    return total_cost

# Sampling a subset of neighbors for efficiency
def sample_neighbors(neighbors, sample_size=7):
    return random.sample(neighbors, sample_size)

# Generating neighbor solutions by swapping stops
def generate_neighbors(route):
    neighbor_solutions = []
    for i in range(len(route)):
        for j in range(i + 1, len(route)):
            new_route = route[:]
            new_route[i], new_route[j] = new_route[j], new_route[i]
            neighbor_solutions.append(new_route)
    return sample_neighbors(neighbor_solutions)

def tabu_search_strategy(initial_route, iterations_limit, tabu_size, initial_stop, departure_time, transport_network, connection_strategy=best_connection_min_transfers):
    search_start_time = tm.time()

    optimal_route = initial_route
    current_route = initial_route
    tabu_registry = []

    for _ in range(iterations_limit):
        neighbor_routes = generate_neighbors(current_route)
        optimal_neighbor = None
        optimal_neighbor_cost = float('inf')

        # Evaluate only non-tabu neighbors
        for neighbor in neighbor_routes:
            if neighbor not in tabu_registry:
                cost = evaluate_solution_cost(neighbor, initial_stop, departure_time, transport_network, connection_strategy)
                if cost < optimal_neighbor_cost:
                    optimal_neighbor = neighbor
                    optimal_neighbor_cost = cost

        # Break if no improvement is found
        if optimal_neighbor is None:
            break

        current_route = optimal_neighbor
        tabu_registry.append(optimal_neighbor)
        if len(tabu_registry) > tabu_size:
            tabu_registry.pop(0)

        # Update optimal route if a better solution is found
        if evaluate_solution_cost(optimal_neighbor, initial_stop, departure_time, transport_network, connection_strategy) < evaluate_solution_cost(optimal_route, initial_stop, departure_time, transport_network, connection_strategy):
            optimal_route = optimal_neighbor

    search_duration = tm.time() - search_start_time
    
    return (optimal_route, solution_costs, search_duration)


In [12]:
import datetime
import sys

start_stop = "PL. GRUNWALDZKI"
departure_time = datetime.time(7, 0, 0)
initial_route = ["Babimojska", "FAT", "Swojczyce", "Biegasa", "Młodych Techników"]
iterations_limit = 6
tabu_size = 10_000

(best_route, solution_costs, search_duration) = tabu_search_strategy(initial_route, iterations_limit, tabu_size, start_stop, departure_time, stop_graph, connection_strategy=best_connection_min_transfers)

print("Best route: {}".format(best_route))

(total_travel_cost, connection_details, astar_routes_dataframes) = get_route_details(best_route, start_stop, departure_time, stop_graph, connection_strategy=best_connection_min_transfers)

best_route.insert(0, start_stop)
best_route.append(start_stop)

i = 0

for optimal_route, route_dataframe in zip(connection_details, astar_routes_dataframes):
    departure_stop = best_route[i]
    arrival_stop = best_route[i+1]
    print("Departure: ", departure_stop, "Arrival: ", arrival_stop)
    route_path = return_path_to_destination(departure_stop, arrival_stop, optimal_route, route_dataframe)
    print_path(route_path)
    i += 1

# Display the total cost and duration of the algorithm execution
print("Total cost: ", seconds_to_hour_and_minute(total_travel_cost), file=sys.stderr)
print("Execution time: ", search_duration, file=sys.stderr)


Best route: ['FAT', 'Babimojska', 'Młodych Techników', 'Biegasa', 'Swojczyce']
Departure:  PL. GRUNWALDZKI Arrival:  FAT
('enter line: D', datetime.time(7, 1), 'at: PL. GRUNWALDZKI')
('leave line: D', datetime.time(7, 11), 'at: Arkady (Capitol)')
('enter line: A', datetime.time(7, 11), 'at: Arkady (Capitol)')
('leave line: A', datetime.time(7, 23), 'at: FAT')
Departure:  FAT Arrival:  Babimojska
('enter line: 126', datetime.time(7, 23), 'at: FAT')
('leave line: 126', datetime.time(7, 31), 'at: Nowodworska')
('enter line: 142', datetime.time(7, 31), 'at: Nowodworska')
('leave line: 13', datetime.time(7, 36), 'at: Babimojska')
Departure:  Babimojska Arrival:  Młodych Techników
('enter line: 13', datetime.time(7, 36), 'at: Babimojska')
('leave line: 13', datetime.time(7, 45), 'at: Młodych Techników')
Departure:  Młodych Techników Arrival:  Biegasa
('enter line: 13', datetime.time(7, 45), 'at: Młodych Techników')
('leave line: 13', datetime.time(7, 55), 'at: GALERIA DOMINIKAŃSKA')
('enter 

Total cost:  ('hours: ', 1, 'minutes: ', 38)
Execution time:  434.7344219684601


In [13]:
import datetime
import sys

initial_stop = "PL. GRUNWALDZKI"
departure_hour = datetime.time(7, 0, 0)
route_plan = ["Babimojska", "FAT", "Swojczyce", "Biegasa", "Młodych Techników"]
iterations_limit = 6
tabu_size = 8

# Performing tabu search to find the best route solution
(optimal_route, evaluated_solutions, search_time) = tabu_search_strategy(route_plan, iterations_limit, tabu_size, initial_stop, departure_hour, stop_graph, connection_strategy=best_connection_min_transfers)

print(f"Best route: {optimal_route}")

# Retrieving detailed information about the best route
(total_cost, route_connections, route_dataframes) = get_route_details(optimal_route, initial_stop, departure_hour, stop_graph, connection_strategy=best_connection_min_transfers)

# Preparing the optimal route for detailed path display
optimal_route.insert(0, initial_stop)
optimal_route.append(initial_stop)

index = 0

# Displaying detailed paths for each leg of the journey within the optimal route
for connection, route_df in zip(route_connections, route_dataframes):
    departure = optimal_route[index]
    arrival = optimal_route[index + 1]
    print(f"Departure: {departure}, Arrival: {arrival}")
    detailed_path = return_path_to_destination(departure, arrival, connection, route_df)
    print_path(detailed_path)
    index += 1

# Printing the total cost and execution time of the algorithm
print(f"Total cost: {seconds_to_hour_and_minute(total_cost)}", file=sys.stderr)
print(f"Execution time: {search_time}", file=sys.stderr)


Best route: ['FAT', 'Babimojska', 'Młodych Techników', 'Biegasa', 'Swojczyce']
Departure: PL. GRUNWALDZKI, Arrival: FAT
('enter line: D', datetime.time(7, 1), 'at: PL. GRUNWALDZKI')
('leave line: D', datetime.time(7, 11), 'at: Arkady (Capitol)')
('enter line: A', datetime.time(7, 11), 'at: Arkady (Capitol)')
('leave line: A', datetime.time(7, 23), 'at: FAT')
Departure: FAT, Arrival: Babimojska
('enter line: 126', datetime.time(7, 23), 'at: FAT')
('leave line: 126', datetime.time(7, 31), 'at: Nowodworska')
('enter line: 142', datetime.time(7, 31), 'at: Nowodworska')
('leave line: 13', datetime.time(7, 36), 'at: Babimojska')
Departure: Babimojska, Arrival: Młodych Techników
('enter line: 13', datetime.time(7, 36), 'at: Babimojska')
('leave line: 13', datetime.time(7, 45), 'at: Młodych Techników')
Departure: Młodych Techników, Arrival: Biegasa
('enter line: 13', datetime.time(7, 45), 'at: Młodych Techników')
('leave line: 13', datetime.time(7, 55), 'at: GALERIA DOMINIKAŃSKA')
('enter line

Total cost: ('hours: ', 1, 'minutes: ', 38)
Execution time: 189.98430585861206


In [14]:
import datetime
import sys

initial_stop = "PL. GRUNWALDZKI"
departure_hour = datetime.time(7, 0, 0)
route_plan = ["Babimojska", "FAT", "Swojczyce", "Biegasa", "Młodych Techników"]
iterations_limit = 6
tabu_size = 8

# Performing tabu search to find the best route solution
(optimal_route, evaluated_solutions, search_time) = tabu_search_strategy(route_plan, iterations_limit, tabu_size, initial_stop, departure_hour, stop_graph, connection_strategy=best_connection_min_time)

print(f"Best route: {optimal_route}")

# Retrieving detailed information about the best route
(total_cost, route_connections, route_dataframes) = get_route_details(optimal_route, initial_stop, departure_hour, stop_graph, connection_strategy=best_connection_min_time)

# Preparing the optimal route for detailed path display
optimal_route.insert(0, initial_stop)
optimal_route.append(initial_stop)

index = 0

# Displaying detailed paths for each leg of the journey within the optimal route
for connection, route_df in zip(route_connections, route_dataframes):
    departure = optimal_route[index]
    arrival = optimal_route[index + 1]
    print(f"Departure: {departure}, Arrival: {arrival}")
    detailed_path = return_path_to_destination(departure, arrival, connection, route_df)
    print_path(detailed_path)
    index += 1

# Printing the total cost and execution time of the algorithm
print(f"Total cost: {seconds_to_hour_and_minute(total_cost)}", file=sys.stderr)
print(f"Execution time: {search_time}", file=sys.stderr)


Best route: ['FAT', 'Babimojska', 'Młodych Techników', 'Biegasa', 'Swojczyce']
Departure: PL. GRUNWALDZKI, Arrival: FAT
('enter line: D', datetime.time(7, 1), 'at: PL. GRUNWALDZKI')
('leave line: D', datetime.time(7, 9), 'at: Renoma')
('enter line: A', datetime.time(7, 9), 'at: Renoma')
('leave line: A', datetime.time(7, 23), 'at: FAT')
Departure: FAT, Arrival: Babimojska
('enter line: 126', datetime.time(7, 23), 'at: FAT')
('leave line: 126', datetime.time(7, 31), 'at: Nowodworska')
('enter line: 142', datetime.time(7, 31), 'at: Nowodworska')
('leave line: 13', datetime.time(7, 36), 'at: Babimojska')
Departure: Babimojska, Arrival: Młodych Techników
('enter line: 13', datetime.time(7, 36), 'at: Babimojska')
('leave line: 13', datetime.time(7, 45), 'at: Młodych Techników')
Departure: Młodych Techników, Arrival: Biegasa
('enter line: 13', datetime.time(7, 45), 'at: Młodych Techników')
('leave line: 13', datetime.time(7, 47), 'at: PL. JANA PAWŁA II')
('enter line: 122', datetime.time(7, 

Total cost: ('hours: ', 1, 'minutes: ', 38)
Execution time: 160.4390070438385


In [15]:
import datetime
import sys

initial_stop = "PL. GRUNWALDZKI"
departure_hour = datetime.time(7, 0, 0)
route_plan = ["Babimojska", "FAT", "Swojczyce", "Biegasa", "Młodych Techników"]
iterations_limit = 6
tabu_size = 4

# Performing tabu search to find the best route solution
(optimal_route, evaluated_solutions, search_time) = tabu_search_strategy(route_plan, iterations_limit, tabu_size, initial_stop, departure_hour, stop_graph, connection_strategy=best_connection_min_transfers)

print(f"Best route: {optimal_route}")

# Retrieving detailed information about the best route
(total_cost, route_connections, route_dataframes) = get_route_details(optimal_route, initial_stop, departure_hour, stop_graph, connection_strategy=best_connection_min_transfers)

# Preparing the optimal route for detailed path display
optimal_route.insert(0, initial_stop)
optimal_route.append(initial_stop)

index = 0

# Displaying detailed paths for each leg of the journey within the optimal route
for connection, route_df in zip(route_connections, route_dataframes):
    departure = optimal_route[index]
    arrival = optimal_route[index + 1]
    print(f"Departure: {departure}, Arrival: {arrival}")
    detailed_path = return_path_to_destination(departure, arrival, connection, route_df)
    print_path(detailed_path)
    index += 1

# Printing the total cost and execution time of the algorithm
print(f"Total cost: {seconds_to_hour_and_minute(total_cost)}", file=sys.stderr)
print(f"Execution time: {search_time}", file=sys.stderr)


Best route: ['FAT', 'Babimojska', 'Młodych Techników', 'Biegasa', 'Swojczyce']
Departure: PL. GRUNWALDZKI, Arrival: FAT
('enter line: D', datetime.time(7, 1), 'at: PL. GRUNWALDZKI')
('leave line: D', datetime.time(7, 11), 'at: Arkady (Capitol)')
('enter line: A', datetime.time(7, 11), 'at: Arkady (Capitol)')
('leave line: A', datetime.time(7, 23), 'at: FAT')
Departure: FAT, Arrival: Babimojska
('enter line: 126', datetime.time(7, 23), 'at: FAT')
('leave line: 126', datetime.time(7, 31), 'at: Nowodworska')
('enter line: 142', datetime.time(7, 31), 'at: Nowodworska')
('leave line: 13', datetime.time(7, 36), 'at: Babimojska')
Departure: Babimojska, Arrival: Młodych Techników
('enter line: 13', datetime.time(7, 36), 'at: Babimojska')
('leave line: 13', datetime.time(7, 45), 'at: Młodych Techników')
Departure: Młodych Techników, Arrival: Biegasa
('enter line: 13', datetime.time(7, 45), 'at: Młodych Techników')
('leave line: 13', datetime.time(7, 55), 'at: GALERIA DOMINIKAŃSKA')
('enter line

Total cost: ('hours: ', 1, 'minutes: ', 38)
Execution time: 10.629596948623657
